In [ ]:
! python -m spacy download en_vectors_web_lg

    32% |██████████▌                     | 217.0MB 17.9MB/s eta 0:00:25

In [11]:
! wget -c --retry-connrefused --tries=0 "https://drive.google.com/uc?export=download&confirm=N84z&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM" -O ~/data/workshop_data/GoogleNews-vectors-negative300.bin

--2019-03-06 13:08:29--  https://drive.google.com/uc?export=download&confirm=N84z&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
Resolving drive.google.com (drive.google.com)... 216.58.204.46, 2a00:1450:4009:80d::200e
Connecting to drive.google.com (drive.google.com)|216.58.204.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/home/jendrik/data/workshop_data/GoogleNews-vectors-negative300.bin’

/home/jendrik/data/     [ <=>                ]   3.21K  --.-KB/s    in 0.001s  

2019-03-06 13:08:30 (5.23 MB/s) - ‘/home/jendrik/data/workshop_data/GoogleNews-vectors-negative300.bin’ saved [136641]



In [13]:
import gensim
from gensim.models import word2vec
import os

In [14]:
home = os.path.expanduser("~")
model = home + '/data/workshop_data/GoogleNews-vectors-negative300.bin'

In [15]:
model = gensim.models.KeyedVectors.load_word2vec_format(model, binary=True)

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431607246399)]

In [16]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951)]

In [17]:
import spacy
from spacy.tokens.token import Token
from numpy import dot
from numpy.linalg import norm

In [18]:
nlp = spacy.load('en_vectors_web_lg')

OSError: [E050] Can't find model 'en_vectors_web_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [19]:
vecs = nlp('king man woman')
out_vec = vecs[0].vector - vecs[1].vector + vecs[2].vector
print(out_vec)

NameError: name 'nlp' is not defined

In [49]:

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

def get_most_similar_word(vec, model):
    print(len(model.vocab))
    # gather all known words, take only the lowercased versions
    allWords = list({w for w in model.vocab if w.has_vector})
    
    allWords.sort(key=lambda w: cosine(w.vector, vec))
    allWords.reverse()
    print("Top 10 most similar words:")
    for word in allWords[:10]:
        print(word.orth_, cosine(word.vector, vec))

In [50]:
get_most_similar_word(out_vec, nlp)

1071393


/home/jendrik/.local/share/virtualenvs/dlworkshop-VuPJaqFs/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in float_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Top 10 most similar words:
fornaro
Post-Teledyne
deferring
THERMAL-PROCESS
LLADRO
sketchers
Otakar
Busacker
Erlandson
kaczorowski
